# Python 数据库操作

本教程将学习Python中如何操作数据库，包括SQLite、MySQL等数据库的使用。

## 1. SQLite 数据库操作

SQLite是一个轻量级的数据库，不需要独立的服务器进程，非常适合学习和小型项目。


In [ ]:
import sqlite3
import os

# 连接到数据库（如果不存在会自动创建）
db_path = "example.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("数据库连接成功！")

# 创建表
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    email TEXT UNIQUE
)
""")

print("表创建成功！")

# 提交更改
conn.commit()
conn.close()
print("数据库连接已关闭")


In [ ]:
# 插入数据
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

# 插入单条数据
cursor.execute("""
INSERT INTO users (name, age, email) 
VALUES (?, ?, ?)
""", ("张三", 25, "zhangsan@example.com"))

# 插入多条数据
users = [
    ("李四", 30, "lisi@example.com"),
    ("王五", 28, "wangwu@example.com"),
    ("赵六", 35, "zhaoliu@example.com")
]

cursor.executemany("""
INSERT INTO users (name, age, email) 
VALUES (?, ?, ?)
""", users)

conn.commit()
print("数据插入成功！")
conn.close()


In [ ]:
# 查询数据
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

# 查询所有数据
cursor.execute("SELECT * FROM users")
all_users = cursor.fetchall()
print("所有用户:")
for user in all_users:
    print(user)

print("\n" + "="*50)

# 查询特定条件的数据
cursor.execute("SELECT * FROM users WHERE age > ?", (28,))
older_users = cursor.fetchall()
print("年龄大于28的用户:")
for user in older_users:
    print(user)

print("\n" + "="*50)

# 查询单条数据
cursor.execute("SELECT * FROM users WHERE email = ?", ("zhangsan@example.com",))
user = cursor.fetchone()
print(f"查询单条数据: {user}")

conn.close()


In [ ]:
# 更新数据
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

# 更新单个字段
cursor.execute("""
UPDATE users 
SET age = ? 
WHERE name = ?
""", (26, "张三"))

# 更新多个字段
cursor.execute("""
UPDATE users 
SET age = ?, email = ? 
WHERE name = ?
""", (31, "newemail@example.com", "李四"))

conn.commit()
print("数据更新成功！")

# 查看更新后的数据
cursor.execute("SELECT * FROM users")
print("\n更新后的数据:")
for user in cursor.fetchall():
    print(user)

conn.close()


In [ ]:
# 删除数据
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

# 删除特定条件的数据
cursor.execute("DELETE FROM users WHERE name = ?", ("赵六",))

conn.commit()
print("数据删除成功！")

# 查看删除后的数据
cursor.execute("SELECT * FROM users")
print("\n删除后的数据:")
for user in cursor.fetchall():
    print(user)

conn.close()


## 2. 使用上下文管理器管理数据库连接


In [ ]:
# 使用上下文管理器（推荐方式）
def get_db_connection(db_path):
    """获取数据库连接（使用上下文管理器）"""
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row  # 让查询结果可以按列名访问
    return conn

# 使用上下文管理器
with get_db_connection("example.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE age > ?", (25,))
    users = cursor.fetchall()
    
    print("使用上下文管理器查询:")
    for user in users:
        # 可以按列名访问
        print(f"ID: {user['id']}, 姓名: {user['name']}, 年龄: {user['age']}")


In [ ]:
# 事务处理（要么全部成功，要么全部失败）
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

try:
    # 开始一个事务（多个操作）
    cursor.execute("UPDATE users SET age = ? WHERE name = ?", (27, "王五"))
    cursor.execute("UPDATE users SET age = ? WHERE name = ?", (32, "李四"))
    
    # 模拟一个错误
    # cursor.execute("UPDATE users SET age = ? WHERE name = ?", ("invalid", "张三"))
    
    # 如果所有操作都成功，提交事务
    conn.commit()
    print("事务提交成功！")
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"发生错误，事务已回滚: {e}")

conn.close()
